In [13]:
import json
import pandas as pd
from datetime import datetime, timedelta
from datetime import date 

Cargamos el dataset tiempo, que nos informa el momento en que se realizo la transaccion. Archivo histórico en el que por cada unidad de tiempo se almacena una lista con los t_id que se han producido en dicha unidad de tiempo

In [2]:
f = open('Datasets/tiempo.json') 
dftiempo = json.load(f) 
f.close()

In [4]:
df_tiempo = pd.DataFrame([key for key in dftiempo.keys()], columns=['hora_transaccion'])

In [5]:
df_tiempo['t_id'] = [value[::] for value in dftiempo.values()]

In [6]:
df_tiempo.sample(5)

,hora_transaccion,t_id
544,545,"[6141290, 6141291, 6141292, 6141293, 6141294, ..."
528,529,"[6117473, 6117474, 6117475, 6117476, 6117477, ..."
678,679,"[6296682, 6296683, 6296684, 6296685, 6296686, ..."
368,369,"[5188066, 5188067, 5188068, 5188069, 5188070, ..."
165,166,"[1906234, 1906235, 1906236, 1906237, 1906238, ..."


In [7]:
df_tiempo_final = df_tiempo.explode('t_id')

In [9]:
df_tiempo_final.sample(3)

,hora_transaccion,t_id
206,207,2570159
179,180,2031738
18,19,453715


In [48]:
df_tiempo_final.shape

(6362620, 2)

La columna hora_transaccion nos indica la cantidad de horas desde el dia 2022-09-01 a las 07:00:00 AM, necesitamos transformar cada fila a formato fecha sumando la hora que corresponde.

In [10]:
df_tiempo_final['hora_transaccion'] = df_tiempo_final['hora_transaccion'].astype(dtype= 'int')

In [11]:
df_tiempo_final.dtypes

hora_transaccion     int32
t_id                object
dtype: object

In [14]:
fecha = datetime(2022, 9, 1, 7,  0, 0)

In [15]:
print(fecha)

2022-09-01 07:00:00


In [16]:
lista_fecha = []
for i in df_tiempo_final['t_id']:
    lista_fecha.append(fecha)

In [38]:
Horas = df_tiempo_final['hora_transaccion']

In [39]:
df_fechas = pd.DataFrame(list(zip(Horas, lista_fecha)), columns = ['hora_transaccion','fecha_origen'])

Utilizamos la función to_timedelta, para convertir la hora en objetos timedelta que están conformados por tantas horas como indica la columna y es posible sumarlos a la fecha.

In [41]:
df_fechas["fecha_final"] = df_fechas["fecha_origen"] + pd.to_timedelta(df_fechas["hora_transaccion"], 'h')

Por último unimos los datasets y solos nos quedamos con la fecha final y el ID de la transacción

In [53]:
df_tiempo_final1 = df_tiempo_final['t_id']

In [55]:
df_tiempo_final2 = pd.DataFrame(df_tiempo_final1)

In [57]:
df_tiempo_final2.tail(5)

,t_id
742,6362615
742,6362616
742,6362617
742,6362618
742,6362619


In [44]:
df_fechas1 = df_fechas['fecha_final']

In [45]:
df_fechas1.tail(5)

6362615   2022-10-02 06:00:00
6362616   2022-10-02 06:00:00
6362617   2022-10-02 06:00:00
6362618   2022-10-02 06:00:00
6362619   2022-10-02 06:00:00
Name: fecha_final, dtype: datetime64[ns]

In [59]:
df_fechas2 = pd.DataFrame(df_fechas1)

In [60]:
df_fechas2.reset_index(level=0, inplace=True)

In [62]:
df_fechas2.tail(5)

,index,fecha_final
6362615,6362615,2022-10-02 06:00:00
6362616,6362616,2022-10-02 06:00:00
6362617,6362617,2022-10-02 06:00:00
6362618,6362618,2022-10-02 06:00:00
6362619,6362619,2022-10-02 06:00:00


In [63]:
df_tiempo3 = pd.merge(df_tiempo_final2, df_fechas2, how= 'inner', left_on = 't_id' , right_on='index') 

In [64]:
df_tiempo3.shape

(6362620, 3)

In [65]:
df_tiempo3.tail(5)

,t_id,index,fecha_final
6362615,6362615,6362615,2022-10-02 06:00:00
6362616,6362616,6362616,2022-10-02 06:00:00
6362617,6362617,6362617,2022-10-02 06:00:00
6362618,6362618,6362618,2022-10-02 06:00:00
6362619,6362619,6362619,2022-10-02 06:00:00


In [66]:
df_tiempo3.drop(['index'], axis=1)

,t_id,fecha_final
0,0,2022-09-01 08:00:00
1,1,2022-09-01 08:00:00
2,2,2022-09-01 08:00:00
3,3,2022-09-01 08:00:00
4,4,2022-09-01 08:00:00
...,...,...
6362615,6362615,2022-10-02 06:00:00
6362616,6362616,2022-10-02 06:00:00
6362617,6362617,2022-10-02 06:00:00
6362618,6362618,2022-10-02 06:00:00


In [67]:
df_tiempo3.to_csv('df_tiempo.csv', index = False)